+++++[link text](https:// [link text](https://))### 1. Read environment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/"Work"/"Original"/"V2_diff_weight_diff_deadline_60_90"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Work/Original/V2_diff_weight_diff_deadline_60_90


In [ ]:
import os

current_directory = os.getcwd()
print("Current Directory:", current_directory)

Current Directory: /content/drive/MyDrive/Work/Original/V2_diff_weight_diff_deadline_60_90


In [ ]:
# ! pip install fastai

In [ ]:
import numpy as np
from mec_environment import MECEnvironment
from random_agent import RandomAgent

mec_evn = MECEnvironment(user_cnt=50, mec_cnt=3,Env_Type=1)

47
{}
26
20
1
{'data_size': 38538, 'circle': 34333, 'user_index': 45, 'priority': 1, 'user_distance': [73.24432832803505, 27.37550383503425, 94.52039350435943], 'delta_max': 0.081}
{'data_size': 7786, 'circle': 126858, 'user_index': 49, 'priority': 2, 'user_distance': [25.21856312829912, 80.65914057569017, 158.66889421232472], 'delta_max': 0.07100000000000001}
{'data_size': 27998, 'circle': 160798, 'user_index': 27, 'priority': 3, 'user_distance': [94.85005783033178, 27.516539712943118, 72.92315438275192], 'delta_max': 0.062}


### 2. Examine the State and Action Spaces

In [ ]:
action_size = mec_evn.action_size()

task=mec_evn.get_task()

states = mec_evn.get_state(task)
print(mec_evn.trans_speed_matrix)
state_size = mec_evn.state_size()
action_space = mec_evn.actions
print('Action size: {}'.format(action_size))
print('Action space: {}'.format(action_space))
print('Observes a state with length: {}'.format(state_size))
print('state size: {}'.format(len(states)))
print('The state for the first agent looks like:', states)

{45: {'speed': array([1.01584111e+05, 2.74989327e+06, 1.99971925e+03])}, 49: {'speed': array([5.51497940e+06, 2.74334686e+05, 1.28396155e+03])}, 27: {'speed': array([  11566.21382718, 6750667.92829696,  194976.75391462])}}
Action size: 45
Action space: [(0, 0.1, 0), (0, 0.1, 1), (0, 0.1, 2), (0, 0.325, 0), (0, 0.325, 1), (0, 0.325, 2), (0, 0.55, 0), (0, 0.55, 1), (0, 0.55, 2), (0, 0.775, 0), (0, 0.775, 1), (0, 0.775, 2), (0, 1.0, 0), (0, 1.0, 1), (0, 1.0, 2), (1, 0.1, 0), (1, 0.1, 1), (1, 0.1, 2), (1, 0.325, 0), (1, 0.325, 1), (1, 0.325, 2), (1, 0.55, 0), (1, 0.55, 1), (1, 0.55, 2), (1, 0.775, 0), (1, 0.775, 1), (1, 0.775, 2), (1, 1.0, 0), (1, 1.0, 1), (1, 1.0, 2), (2, 0.1, 0), (2, 0.1, 1), (2, 0.1, 2), (2, 0.325, 0), (2, 0.325, 1), (2, 0.325, 2), (2, 0.55, 0), (2, 0.55, 1), (2, 0.55, 2), (2, 0.775, 0), (2, 0.775, 1), (2, 0.775, 2), (2, 1.0, 0), (2, 1.0, 1), (2, 1.0, 2)]
Observes a state with length: 30
state size: 30
The state for the first agent looks like: [1.11054277e-02 6.02911264

### 3. Take Random Actions in the Environment

### 6. Take Actions with DQN

In [ ]:
epoch_no =2500
window_size =250

In [ ]:
import torch
from collections import deque
from greedy_agent import GreedyAgent
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
from dqn_agent import Agent
agent = Agent(state_size=state_size,action_size=action_size,seed=0)
model_path = os.path.join(current_directory, 'model_dqnp_checkpoint.pth')


In [ ]:
def dqnp(n_episodes=epoch_no, max_t=1000, eps_start=1.0, eps_end=0.001, eps_decay=0.995,bench_score=1500000):
    """Deep Q-Learning.

    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of atimesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    epi_scores = []                        # list containing scores from each episode
    epi_task=[]
    epi_task_P1=[]
    epi_task_P2=[]
    epi_task_P3=[]


    epi_latency=[]
    epi_latency_P1=[]
    epi_latency_P2=[]
    epi_latency_P3=[]
    epi_task_select_P1=[]
    epi_task_select_P2=[]
    epi_task_select_P3=[]
    epi_datasize=[]
    epi_datasize_P1=[]
    epi_datasize_P2=[]
    epi_datasize_P3=[]

    epi_energy=[]
    epi_energy_P1=[]
    epi_energy_P2=[]
    epi_energy_P3=[]

    epi_num_task=[]
    epi_num_task_P1=[]
    epi_num_task_P2=[]
    epi_num_task_P3=[]

    mean_scores = [] # average score by the windos size
    mean_task=[] # average task ratio by window size
    mean_energy = [] # average energy cost by window size
    mean_latency=[]
    mean_datasize=[]

    mean_tasks_P1=[]
    mean_tasks_P2=[]
    mean_tasks_P3=[]

    mean_energy_P1=[]
    mean_energy_P2=[]
    mean_energy_P3=[]

    mean_latency_P1=[]
    mean_latency_P2=[]
    mean_latency_P3=[]


    mean_datasize_P1=[]
    mean_datasize_P2=[]
    mean_datasize_P3=[]
    reward_list_P1=[]
    reward_list_P2=[]
    reward_list_P3=[]

    scores_window = deque(maxlen=window_size)  # last score_window_size scores
    task_done_ratio_window = deque(maxlen=window_size)  # last  task dones
    energy_window = deque(maxlen=window_size)  # last  engery cost
    latency_window = deque(maxlen=window_size)  # last  engery cost
    datasize_window= deque(maxlen=window_size)


    mean_tasks_P1_window = deque(maxlen=window_size)  # last  engery cost
    mean_tasks_P2_window = deque(maxlen=window_size)  # last  engery cost
    mean_tasks_P3_window = deque(maxlen=window_size)  # last  engery cost

    win_num_task = deque(maxlen=window_size)
    win_num_task_P1 = deque(maxlen=window_size)  # last  engery cost
    win_num_task_P2 = deque(maxlen=window_size)  # last  engery cost
    win_num_task_P3 = deque(maxlen=window_size)  # last  engery cost

    mean_datasize_window=deque(maxlen=window_size)
    mean_datasize_P1_window= deque(maxlen=window_size)  # last  engery cost
    mean_datasize_P2_window = deque(maxlen=window_size)  # last  engery cost
    mean_datasize_P3_window = deque(maxlen=window_size)  # last  engery cost

    mean_energy_window_P1 = deque(maxlen=window_size)  # last  engery cost
    mean_energy_window_P2 = deque(maxlen=window_size)  # last  engery cost
    mean_energy_window_P3 = deque(maxlen=window_size)  # last  engery cost

    mean_latency_P1_window=deque(maxlen=window_size)
    mean_latency_P2_window=deque(maxlen=window_size)
    mean_latency_P3_window=deque(maxlen=window_size)



    eps = eps_start                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        mec_evn.reset() # reset the environment

        score = 0
        energy = []
        latency=[]
        task_done = []
        energy_P1=[]
        energy_P2=[]
        energy_P3=[]
        task_done_P1=[]
        task_done_P2=[]
        task_done_P3=[]
        latency_P1=[]
        latency_P2=[]
        latency_P3=[]
        datasize=[]
        datasize_P1=[]
        datasize_P2=[]
        datasize_P3=[]



        tasks = mec_evn.get_task()
        for t in range(max_t):

            state = mec_evn.get_state(tasks)
            action = agent.act(state, eps)
            next_state, feedback,done,action_task,next_task,rec_mec_idx = mec_evn.step(action,tasks)
            agent.step(state, action, feedback['reward'], next_state, done)

            score += feedback['reward']
            energy.append(feedback['energy'])
            datasize.append(action_task['data_size'])

            if feedback['task_done']==1:

              latency.append(feedback['latency'])


            if action_task['data_size'] != 0:
              task_done.append(feedback['task_done'])
              if action_task['priority']==1:
                  energy_P1.append(feedback['energy'])
                  task_done_P1.append(feedback['task_done'])
                  datasize_P1.append(action_task['data_size'])
                  reward_list_P1.append(feedback['reward'])

                  if feedback['task_done']:
                    latency_P1.append(feedback['latency'])

              if action_task['priority']==2:
                  energy_P2.append(feedback['energy'])
                  task_done_P2.append(feedback['task_done']*feedback['task_done'])
                  datasize_P2.append(action_task['data_size'])
                  reward_list_P2.append(feedback['reward'])
                  if feedback['task_done']:
                    latency_P2.append(feedback['latency'])

              if action_task['priority']==3:
                  energy_P3.append(feedback['energy'])
                  task_done_P3.append(feedback['task_done'])
                  datasize_P3.append(action_task['data_size'])
                  reward_list_P3.append(feedback['reward'])
                  if feedback['task_done']:
                    latency_P3.append(feedback['latency'])



            tasks=next_task
            if done:
                break
        scores_window.append(score)       # save most recent score
        epi_scores.append(score)              # save most recent score
        mean_score = np.mean(scores_window)
        epi_task.append(np.mean(task_done))
        epi_task_P1.append(np.mean(task_done_P1))
        epi_task_P2.append(np.mean(task_done_P2))
        epi_task_P3.append(np.mean(task_done_P3))

        epi_task_select_P1.append(len(task_done_P1))
        epi_task_select_P2.append(len(task_done_P2))
        epi_task_select_P3.append(len(task_done_P3))

        epi_latency.append(np.mean(latency))
        epi_latency_P1.append(np.mean(latency_P1))
        epi_latency_P2.append(np.mean(latency_P2))
        epi_latency_P3.append(np.mean(latency_P3))

        epi_energy.append(np.mean(energy))
        epi_energy_P1.append(np.mean(energy_P1))
        epi_energy_P2.append(np.mean(energy_P2))
        epi_energy_P3.append(np.mean(energy_P3))

        epi_num_task.append(sum(task_done_P1)+sum(task_done_P2)+sum(task_done_P3))
        epi_num_task_P1.append(sum(task_done_P1))
        epi_num_task_P2.append(sum(task_done_P2))
        epi_num_task_P3.append(sum(task_done_P3))

        epi_datasize.append(np.mean(datasize))
        epi_datasize_P1.append(np.mean(datasize_P1))
        epi_datasize_P2.append(np.mean(datasize_P2))
        epi_datasize_P3.append(np.mean(datasize_P3))





        eps = max(eps_end, eps_decay*eps) # decrease epsilon

        print('\rEpisode {}\tAverage Score: {:.4f}\tmeantaskP1: {:.4f}\tmeantaskP2: {:.4f}\tmeantaskP3: {:.4f}\tmeanenergyP1: \
            {:.4f}\tmeanenergyP2: {:.4f}\tmeanenergyP3: {:.4f}\tmeanlatencyP1: {:.4f}\tmeanlatencyP2: {:.4f}\tmeanlatencyP3:{:.4f}'.format(i_episode, \
            mean_score, np.mean(task_done_P1), np.mean(task_done_P2), np.mean(task_done_P3), np.mean(energy_P1), np.mean(energy_P2), np.mean(energy_P3), \
                                    np.mean(latency_P1), np.mean(latency_P2), np.mean(latency_P3)), end="")





        if i_episode % window_size == 0:


            mean_scores.append(mean_score)
            task_done_ratio_window.append(np.mean(task_done))       # task done ratio
            mean_task.append(np.mean(task_done_ratio_window))
            energy_window.append(np.mean(energy))
            latency_window.append(np.mean(latency))

            mean_energy.append(np.mean(energy_window))
            mean_latency.append(np.mean(latency_window))


            mean_datasize_window.append(np.mean(datasize))
            mean_datasize.append(np.mean(mean_datasize_window))

            mean_tasks_P1_window.append(np.mean(task_done_P1))
            mean_tasks_P1.append(np.mean( mean_tasks_P1_window))

            mean_tasks_P2_window.append(np.mean(task_done_P2))
            mean_tasks_P2.append(np.mean( mean_tasks_P2_window))

            mean_tasks_P3_window.append(np.mean(task_done_P3))
            mean_tasks_P3.append(np.mean( mean_tasks_P3_window))

            mean_energy_window_P1.append(np.mean(energy_P1))
            mean_energy_P1.append(np.mean(mean_energy_window_P1))

            mean_energy_window_P2.append(np.mean(energy_P2))
            mean_energy_P2.append(np.mean(mean_energy_window_P2))

            mean_energy_window_P3.append(np.mean(energy_P3))
            mean_energy_P3.append(np.mean(mean_energy_window_P3))


            mean_latency_P1_window.append(np.mean(latency_P1))
            mean_latency_P1.append(np.mean( mean_latency_P1_window))

            mean_latency_P2_window.append(np.mean(latency_P2))
            mean_latency_P2.append(np.mean( mean_latency_P2_window))

            mean_latency_P3_window.append(np.mean(latency_P3))
            mean_latency_P3.append(np.mean( mean_latency_P3_window))

            mean_datasize_P1_window.append(np.mean(datasize_P1))
            mean_datasize_P1.append(np.mean(mean_datasize_P1_window))

            mean_datasize_P2_window.append(np.mean(datasize_P2))
            mean_datasize_P2.append(np.mean(mean_datasize_P2_window))

            mean_datasize_P3_window.append(np.mean(datasize_P3))
            mean_datasize_P3.append(np.mean(mean_datasize_P3_window))


            win_num_task.append(sum(task_done_P1)+sum(task_done_P2)+sum(task_done_P3))
            win_num_task_P1.append(sum(task_done_P1))
            win_num_task_P2.append(sum(task_done_P2))
            win_num_task_P3.append(sum(task_done_P3))


            print('\rEpisode {}\tAverage Score: {:.4f}\tmeantaskP1: {:.4f}\tmeantaskP2: {:.4f}\tmeantaskP3: {:.4f}\tmeanenergyP1: \
            {:.4f}\tmeanenergyP2: {:.4f}\tmeanenergyP3: {:.4f} \tmeanlatencyP1: {:.4f} \tmeanlatencyP2: {:.4f} \tmeanlatencyP3: {:.4f} \tmeandatasize: {:.4f} \tmeandatasizeP1: {:.4f} \tmeandatasizeP2: {:.4f} \tmeandatasizeP3: {:.4f}' .format(i_episode, mean_score,\
            mean_tasks_P1[-1],mean_tasks_P2[-1],mean_tasks_P3[-1],mean_energy_P1[-1],mean_energy_P2[-1],mean_energy_P3[-1], mean_latency_P1[-1],mean_latency_P2[-1],mean_latency_P3[-1],mean_datasize[-1],mean_datasize_P1[-1], \
                                                                                                                          mean_datasize_P2[-1],mean_datasize_P3[-1]



                                                                                                                                                                                                                                                       ))


            #print(len(task_done_P1),len(task_done_P2),len(task_done_P3))
           # print(len(task_done_P1)+len(task_done_P2)+len(task_done_P3))
            print('mean_task_till_this_window',np.mean(epi_task[-window_size:]), 'T1::',   np.mean(epi_task_P1[-window_size:]) ,'T2::' ,np.mean(epi_task_P2[-window_size:]) ,'T3::',np.mean(epi_task_P3[-window_size:]))
            #print('mean_energy_till_this_window',np.mean(mean_energy[-window_size:]), 'T1::', np.mean(mean_energy_P1[-window_size:])  ,'T2::', np.mean(mean_energy_P2[-window_size:])  ,'T3::',np.mean(mean_energy_P3[-window_size:]))
           # print('mean Latency till this window',np.mean(mean_latency[-window_size:]),'T1::', np.mean(mean_latency_P1[-window_size:]),'T1::', np.mean(mean_latency_P2[-window_size:]),'T1::', np.mean(mean_latency_P3[-window_size:]))
           # print(type(mean_tasks_P1),type(epi_num_task_P1))
            #print('mean_datasize', mean_datasize[-1])
            print( 'task_select_P1', np.mean(epi_task_select_P1[-window_size:]), 'task_select_P2', np.mean(epi_task_select_P2[-window_size:]),'task_select_P3', np.mean(epi_task_select_P3[-window_size:]))
            print('total_task_done_P1::', np.mean(epi_num_task_P1[-window_size:]),'total_task_done_P2::', np.mean(epi_num_task_P2[-window_size:]),'total_task_done_P3::',np.mean(epi_num_task_P3[-window_size:]))

            #print(np.mean(reward_list_P1), np.mean(reward_list_P2), np.mean(reward_list_P3))
            #print(min(reward_list_P1), min(reward_list_P2), min(reward_list_P3))
           # print(max(reward_list_P1), max(reward_list_P2), max(reward_list_P3))


            #print(mean_tasks_P1)
           # print(epi_task_P1)



        if np.mean(scores_window)>=bench_score or i_episode % n_episodes==0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.4f}'.format(i_episode, mean_score))
            torch.save(agent.qnetwork_local.state_dict(), model_path)
            break

    return epi_scores,epi_task,epi_energy,epi_latency,epi_num_task, epi_task_P1,epi_task_P2,epi_task_P3,epi_latency_P1,epi_latency_P2,epi_latency_P3,epi_energy_P1,epi_energy_P2,epi_energy_P3,epi_num_task_P1,epi_num_task_P2,epi_num_task_P3, \
    epi_task_select_P1,epi_task_select_P2,epi_task_select_P3,mean_scores,mean_task,mean_energy,mean_latency,win_num_task, mean_tasks_P1,mean_tasks_P2,mean_tasks_P3, mean_energy_P1,\
                 mean_energy_P2, mean_energy_P3,mean_latency_P1,mean_latency_P2,mean_latency_P3,win_num_task_P1,win_num_task_P2,win_num_task_P3,epi_datasize,epi_datasize_P1,epi_datasize_P2,epi_datasize_P3,\
                 mean_datasize,mean_datasize_P1,mean_datasize_P2,mean_datasize_P3

dqnp_epi_scores,dqnp_epi_task,dqnp_epi_energy,dqnp_epi_latency,dqnp_epi_num_task,dqnp_epi_task_P1,dqnp_epi_task_P2,dqnp_epi_task_P3,dqnp_epi_latency_P1,dqnp_epi_latency_P2,dqnp_epi_latency_P3,\
dqnp_epi_energy_P1,dqnp_epi_energy_P2,dqnp_epi_energy_P3,dqnp_epi_num_task_P1,dqnp_epi_num_task_P2,dqnp_epi_num_task_P3,dqnp_epi_task_select_P1,dqnp_epi_task_select_P2,dqnp_epi_task_select_P3,\
dqnp_mean_scores,dqnp_mean_task,dqnp_mean_energy,dqnp_mean_latency,dqnp_win_num_task, dqnp_mean_tasks_P1,dqnp_mean_tasks_P2,dqnp_mean_tasks_P3,\
dqnp_mean_energy_P1,dqnp_mean_energy_P2,dqnp_mean_energy_P3,dqnp_mean_latency_P1,dqnp_mean_latency_P2,dqnp_mean_latency_P3 ,dqnp_win_num_task_P1, dqnp_win_num_task_P2,dqnp_win_num_task_P3,\
dqnp_epi_datasize,dqnp_epi_datasize_P1,dqnp_epi_datasize_P2,dqnp_epi_datasize_P3,dqnp_mean_datasize,dqnp_mean_datasize_P1,dqnp_mean_datasize_P2,dqnp_mean_datasize_P3 = dqnp()

Episode 250	Average Score: 329.4058	meantaskP1: 0.7645	meantaskP2: 0.7794	meantaskP3: 0.8763	meanenergyP1:             0.1938	meanenergyP2: 0.1693	meanenergyP3: 0.6793 	meanlatencyP1: 0.0150 	meanlatencyP2: 0.0140 	meanlatencyP3: 0.0113 	meandatasize: 22100.6380 	meandatasizeP1: 24995.1240 	meandatasizeP2: 23455.5302 	meandatasizeP3: 19833.9916
mean_task_till_this_window 0.6169831108442679 T1:: 0.5810660514504425 T2:: 0.59623054523287 T3:: 0.6566013296512307
task_select_P1 284.452 task_select_P2 304.324 task_select_P3 393.888
total_task_done_P1:: 163.08 total_task_done_P2:: 180.632 total_task_done_P3:: 263.376
Episode 500	Average Score: 474.1415	meantaskP1: 0.8358	meantaskP2: 0.8373	meantaskP3: 0.9122	meanenergyP1:             0.1614	meanenergyP2: 0.1142	meanenergyP3: 0.3526 	meanlatencyP1: 0.0137 	meanlatencyP2: 0.0122 	meanlatencyP3: 0.0112 	meandatasize: 21956.8610 	meandatasizeP1: 24525.0053 	meandatasizeP2: 23400.1260 	meandatasizeP3: 20093.4698
mean_task_till_this_window 0.880413

In [ ]:
import csv


# Combining data into a list of tuples
data = zip(range(1, len(dqnp_epi_scores) + 1), dqnp_epi_scores,dqnp_epi_task,dqnp_epi_energy,dqnp_epi_latency,dqnp_epi_num_task,dqnp_epi_task_P1,dqnp_epi_task_P2,
           dqnp_epi_task_P3,dqnp_epi_latency_P1,dqnp_epi_latency_P2,dqnp_epi_latency_P3,
           dqnp_epi_energy_P1,dqnp_epi_energy_P2,dqnp_epi_energy_P3,dqnp_epi_num_task_P1,dqnp_epi_num_task_P2,dqnp_epi_num_task_P3,
           dqnp_epi_datasize,dqnp_epi_datasize_P1,dqnp_epi_datasize_P2,dqnp_epi_datasize_P3,dqnp_epi_task_select_P1,dqnp_epi_task_select_P2,dqnp_epi_task_select_P3)

# Writing data to a CSV file
with open("data_dqnp.csv", mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Index","dqnp_epi_scores","dqnp_epi_task","dqnp_epi_energy","dqnp_epi_latency","dqnp_epi_num_task","dqnp_epi_task_P1","dqnp_epi_task_P2","dqnp_epi_task_P3",
                     "dqnp_epi_latency_P1","dqnp_epi_latency_P2","dqnp_epi_latency_P3","dqnp_epi_energy_P1","dqnp_epi_energy_P2","dqnp_epi_energy_P3","dqnp_epi_num_task_P1",
                     "dqnp_epi_num_task_P2","dqnp_epi_num_task_P3","dqnp_epi_datasize","dqnp_epi_datasize_P1","dqnp_epi_datasize_P2"
                     "dqnp_epi_datasize_P3","dqnp_epi_task_select_P1","dqnp_epi_task_select_P2","dqnp_epi_task_select_P3"])  # Writing header
    writer.writerows(data)  # Writing data rows


In [ ]:
import os

current_directory = os.getcwd()
print("Current Directory:", current_directory)

In [ ]:
import numpy as np
from mec_environment import MECEnvironment
from random_agent import RandomAgent

mec_evn = MECEnvironment(user_cnt=50, mec_cnt=3,Env_Type=3)

In [ ]:
action_size = mec_evn.action_size()
Task=mec_evn.get_task()
states = mec_evn.get_state(Task)
state_size = mec_evn.state_size()
action_space = mec_evn.actions
print('Action size: {}'.format(action_size))
print('Action space: {}'.format(action_space))
print('Observes a state with length: {}'.format(state_size))
print('state size: {}'.format(len(states)))
print('The state for the first agent looks like:', states)

In [ ]:
epoch_no =2500
window_size =250

In [ ]:
import torch
from collections import deque
from greedy_agent import GreedyAgent
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
from dqn_agent import Agent
agent = Agent(state_size=state_size,action_size=action_size,seed=0)
model_path = os.path.join(current_directory, 'model_dqnb_checkpoint.pth')

In [ ]:
def dqnb(n_episodes=epoch_no, max_t=1000, eps_start=1.0, eps_end=0.001, eps_decay=0.995,bench_score=1500000):
    """Deep Q-Learning.

    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of atimesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    epi_scores = []                        # list containing scores from each episode
    epi_task=[]
    epi_task_P1=[]
    epi_task_P2=[]
    epi_task_P3=[]


    epi_latency=[]
    epi_latency_P1=[]
    epi_latency_P2=[]
    epi_latency_P3=[]
    epi_task_select_P1=[]
    epi_task_select_P2=[]
    epi_task_select_P3=[]
    epi_datasize=[]
    epi_datasize_P1=[]
    epi_datasize_P2=[]
    epi_datasize_P3=[]

    epi_energy=[]
    epi_energy_P1=[]
    epi_energy_P2=[]
    epi_energy_P3=[]

    epi_num_task=[]
    epi_num_task_P1=[]
    epi_num_task_P2=[]
    epi_num_task_P3=[]

    mean_scores = [] # average score by the windos size
    mean_task=[] # average task ratio by window size
    mean_energy = [] # average energy cost by window size
    mean_latency=[]
    mean_datasize=[]

    mean_tasks_P1=[]
    mean_tasks_P2=[]
    mean_tasks_P3=[]

    mean_energy_P1=[]
    mean_energy_P2=[]
    mean_energy_P3=[]

    mean_latency_P1=[]
    mean_latency_P2=[]
    mean_latency_P3=[]


    mean_datasize_P1=[]
    mean_datasize_P2=[]
    mean_datasize_P3=[]
    reward_list_P1=[]
    reward_list_P2=[]
    reward_list_P3=[]

    scores_window = deque(maxlen=window_size)  # last score_window_size scores
    task_done_ratio_window = deque(maxlen=window_size)  # last  task dones
    energy_window = deque(maxlen=window_size)  # last  engery cost
    latency_window = deque(maxlen=window_size)  # last  engery cost
    datasize_window= deque(maxlen=window_size)


    mean_tasks_P1_window = deque(maxlen=window_size)  # last  engery cost
    mean_tasks_P2_window = deque(maxlen=window_size)  # last  engery cost
    mean_tasks_P3_window = deque(maxlen=window_size)  # last  engery cost

    win_num_task = deque(maxlen=window_size)
    win_num_task_P1 = deque(maxlen=window_size)  # last  engery cost
    win_num_task_P2 = deque(maxlen=window_size)  # last  engery cost
    win_num_task_P3 = deque(maxlen=window_size)  # last  engery cost

    mean_datasize_window=deque(maxlen=window_size)
    mean_datasize_P1_window= deque(maxlen=window_size)  # last  engery cost
    mean_datasize_P2_window = deque(maxlen=window_size)  # last  engery cost
    mean_datasize_P3_window = deque(maxlen=window_size)  # last  engery cost

    mean_energy_window_P1 = deque(maxlen=window_size)  # last  engery cost
    mean_energy_window_P2 = deque(maxlen=window_size)  # last  engery cost
    mean_energy_window_P3 = deque(maxlen=window_size)  # last  engery cost

    mean_latency_P1_window=deque(maxlen=window_size)
    mean_latency_P2_window=deque(maxlen=window_size)
    mean_latency_P3_window=deque(maxlen=window_size)



    eps = eps_start                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        mec_evn.reset() # reset the environment

        score = 0
        energy = []
        latency=[]
        task_done = []
        energy_P1=[]
        energy_P2=[]
        energy_P3=[]
        task_done_P1=[]
        task_done_P2=[]
        task_done_P3=[]
        latency_P1=[]
        latency_P2=[]
        latency_P3=[]
        datasize=[]
        datasize_P1=[]
        datasize_P2=[]
        datasize_P3=[]



        tasks = mec_evn.get_task()
        for t in range(max_t):

            state = mec_evn.get_state(tasks)
            action = agent.act(state, eps)
            next_state, feedback,done,action_task,next_task = mec_evn.step(action,tasks)
            agent.step(state, action, feedback['reward'], next_state, done)

            score += feedback['reward']
            energy.append(feedback['energy'])
            datasize.append(action_task['data_size'])

            if feedback['task_done']==1:

              latency.append(feedback['latency'])


            if action_task['data_size'] != 0:
              task_done.append(feedback['task_done'])
              if action_task['priority']==1:
                  energy_P1.append(feedback['energy'])
                  task_done_P1.append(feedback['task_done'])
                  datasize_P1.append(action_task['data_size'])
                  reward_list_P1.append(feedback['reward'])

                  if feedback['task_done']:
                    latency_P1.append(feedback['latency'])

              if action_task['priority']==2:
                  energy_P2.append(feedback['energy'])
                  task_done_P2.append(feedback['task_done']*feedback['task_done'])
                  datasize_P2.append(action_task['data_size'])
                  reward_list_P2.append(feedback['reward'])
                  if feedback['task_done']:
                    latency_P2.append(feedback['latency'])

              if action_task['priority']==3:
                  energy_P3.append(feedback['energy'])
                  task_done_P3.append(feedback['task_done'])
                  datasize_P3.append(action_task['data_size'])
                  reward_list_P3.append(feedback['reward'])
                  if feedback['task_done']:
                    latency_P3.append(feedback['latency'])



            tasks=next_task
            if done:
                break
        scores_window.append(score)       # save most recent score
        epi_scores.append(score)              # save most recent score
        mean_score = np.mean(scores_window)
        epi_task.append(np.mean(task_done))
        epi_task_P1.append(np.mean(task_done_P1))
        epi_task_P2.append(np.mean(task_done_P2))
        epi_task_P3.append(np.mean(task_done_P3))

        epi_task_select_P1.append(len(task_done_P1))
        epi_task_select_P2.append(len(task_done_P2))
        epi_task_select_P3.append(len(task_done_P3))

        epi_latency.append(np.mean(latency))
        epi_latency_P1.append(np.mean(latency_P1))
        epi_latency_P2.append(np.mean(latency_P2))
        epi_latency_P3.append(np.mean(latency_P3))

        epi_energy.append(np.mean(energy))
        epi_energy_P1.append(np.mean(energy_P1))
        epi_energy_P2.append(np.mean(energy_P2))
        epi_energy_P3.append(np.mean(energy_P3))

        epi_num_task.append(sum(task_done_P1)+sum(task_done_P2)+sum(task_done_P3))
        epi_num_task_P1.append(sum(task_done_P1))
        epi_num_task_P2.append(sum(task_done_P2))
        epi_num_task_P3.append(sum(task_done_P3))

        epi_datasize.append(np.mean(datasize))
        epi_datasize_P1.append(np.mean(datasize_P1))
        epi_datasize_P2.append(np.mean(datasize_P2))
        epi_datasize_P3.append(np.mean(datasize_P3))





        eps = max(eps_end, eps_decay*eps) # decrease epsilon

        print('\rEpisode {}\tAverage Score: {:.4f}\tmeantaskP1: {:.4f}\tmeantaskP2: {:.4f}\tmeantaskP3: {:.4f}\tmeanenergyP1: \
            {:.4f}\tmeanenergyP2: {:.4f}\tmeanenergyP3: {:.4f}\tmeanlatencyP1: {:.4f}\tmeanlatencyP2: {:.4f}\tmeanlatencyP3:{:.4f}'.format(i_episode, \
            mean_score, np.mean(task_done_P1), np.mean(task_done_P2), np.mean(task_done_P3), np.mean(energy_P1), np.mean(energy_P2), np.mean(energy_P3), \
                                    np.mean(latency_P1), np.mean(latency_P2), np.mean(latency_P3)), end="")





        if i_episode % window_size == 0:


            mean_scores.append(mean_score)
            task_done_ratio_window.append(np.mean(task_done))       # task done ratio
            mean_task.append(np.mean(task_done_ratio_window))
            energy_window.append(np.mean(energy))
            latency_window.append(np.mean(latency))

            mean_energy.append(np.mean(energy_window))
            mean_latency.append(np.mean(latency_window))


            mean_datasize_window.append(np.mean(datasize))
            mean_datasize.append(np.mean(mean_datasize_window))

            mean_tasks_P1_window.append(np.mean(task_done_P1))
            mean_tasks_P1.append(np.mean( mean_tasks_P1_window))

            mean_tasks_P2_window.append(np.mean(task_done_P2))
            mean_tasks_P2.append(np.mean( mean_tasks_P2_window))

            mean_tasks_P3_window.append(np.mean(task_done_P3))
            mean_tasks_P3.append(np.mean( mean_tasks_P3_window))

            mean_energy_window_P1.append(np.mean(energy_P1))
            mean_energy_P1.append(np.mean(mean_energy_window_P1))

            mean_energy_window_P2.append(np.mean(energy_P2))
            mean_energy_P2.append(np.mean(mean_energy_window_P2))

            mean_energy_window_P3.append(np.mean(energy_P3))
            mean_energy_P3.append(np.mean(mean_energy_window_P3))


            mean_latency_P1_window.append(np.mean(latency_P1))
            mean_latency_P1.append(np.mean( mean_latency_P1_window))

            mean_latency_P2_window.append(np.mean(latency_P2))
            mean_latency_P2.append(np.mean( mean_latency_P2_window))

            mean_latency_P3_window.append(np.mean(latency_P3))
            mean_latency_P3.append(np.mean( mean_latency_P3_window))

            mean_datasize_P1_window.append(np.mean(datasize_P1))
            mean_datasize_P1.append(np.mean(mean_datasize_P1_window))

            mean_datasize_P2_window.append(np.mean(datasize_P2))
            mean_datasize_P2.append(np.mean(mean_datasize_P2_window))

            mean_datasize_P3_window.append(np.mean(datasize_P3))
            mean_datasize_P3.append(np.mean(mean_datasize_P3_window))


            win_num_task.append(sum(task_done_P1)+sum(task_done_P2)+sum(task_done_P3))
            win_num_task_P1.append(sum(task_done_P1))
            win_num_task_P2.append(sum(task_done_P2))
            win_num_task_P3.append(sum(task_done_P3))


            print('\rEpisode {}\tAverage Score: {:.4f}\tmeantaskP1: {:.4f}\tmeantaskP2: {:.4f}\tmeantaskP3: {:.4f}\tmeanenergyP1: \
            {:.4f}\tmeanenergyP2: {:.4f}\tmeanenergyP3: {:.4f} \tmeanlatencyP1: {:.4f} \tmeanlatencyP2: {:.4f} \tmeanlatencyP3: {:.4f} \tmeandatasize: {:.4f} \tmeandatasizeP1: {:.4f} \tmeandatasizeP2: {:.4f} \tmeandatasizeP3: {:.4f}' .format(i_episode, mean_score,\
            mean_tasks_P1[-1],mean_tasks_P2[-1],mean_tasks_P3[-1],mean_energy_P1[-1],mean_energy_P2[-1],mean_energy_P3[-1], mean_latency_P1[-1],mean_latency_P2[-1],mean_latency_P3[-1],mean_datasize[-1],mean_datasize_P1[-1], \
                                                                                                                          mean_datasize_P2[-1],mean_datasize_P3[-1]



                                                                                                                                                                                                                                                       ))


            #print(len(task_done_P1),len(task_done_P2),len(task_done_P3))
           # print(len(task_done_P1)+len(task_done_P2)+len(task_done_P3))
            print('mean_task_till_this_window',np.mean(epi_task[-window_size:]), 'T1::',   np.mean(epi_task_P1[-window_size:]) ,'T2::' ,np.mean(epi_task_P2[-window_size:]) ,'T3::',np.mean(epi_task_P3[-window_size:]))
            #print('mean_energy_till_this_window',np.mean(mean_energy[-window_size:]), 'T1::', np.mean(mean_energy_P1[-window_size:])  ,'T2::', np.mean(mean_energy_P2[-window_size:])  ,'T3::',np.mean(mean_energy_P3[-window_size:]))
           # print('mean Latency till this window',np.mean(mean_latency[-window_size:]),'T1::', np.mean(mean_latency_P1[-window_size:]),'T1::', np.mean(mean_latency_P2[-window_size:]),'T1::', np.mean(mean_latency_P3[-window_size:]))
           # print(type(mean_tasks_P1),type(epi_num_task_P1))
            #print('mean_datasize', mean_datasize[-1])
            print( 'task_select_P1', np.mean(epi_task_select_P1[-window_size:]), 'task_select_P2', np.mean(epi_task_select_P2[-window_size:]),'task_select_P3', np.mean(epi_task_select_P3[-window_size:]))
            print('total_task_done_P1::', np.mean(epi_num_task_P1[-window_size:]),'total_task_done_P2::', np.mean(epi_num_task_P2[-window_size:]),'total_task_done_P3::',np.mean(epi_num_task_P3[-window_size:]))

            #print(np.mean(reward_list_P1), np.mean(reward_list_P2), np.mean(reward_list_P3))
            #print(min(reward_list_P1), min(reward_list_P2), min(reward_list_P3))
           # print(max(reward_list_P1), max(reward_list_P2), max(reward_list_P3))


            #print(mean_tasks_P1)
           # print(epi_task_P1)



        if np.mean(scores_window)>=bench_score or i_episode % n_episodes==0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.4f}'.format(i_episode, mean_score))
            torch.save(agent.qnetwork_local.state_dict(), model_path)
            break

    return epi_scores,epi_task,epi_energy,epi_latency,epi_num_task, epi_task_P1,epi_task_P2,epi_task_P3,epi_latency_P1,epi_latency_P2,epi_latency_P3,epi_energy_P1,epi_energy_P2,epi_energy_P3,epi_num_task_P1,epi_num_task_P2,epi_num_task_P3, \
    epi_task_select_P1,epi_task_select_P2,epi_task_select_P3,mean_scores,mean_task,mean_energy,mean_latency,win_num_task, mean_tasks_P1,mean_tasks_P2,mean_tasks_P3, mean_energy_P1,\
                 mean_energy_P2, mean_energy_P3,mean_latency_P1,mean_latency_P2,mean_latency_P3,win_num_task_P1,win_num_task_P2,win_num_task_P3,epi_datasize,epi_datasize_P1,epi_datasize_P2,epi_datasize_P3,\
                 mean_datasize,mean_datasize_P1,mean_datasize_P2,mean_datasize_P3

dqnb_epi_scores,dqnb_epi_task,dqnb_epi_energy,dqnb_epi_latency,dqnb_epi_num_task,dqnb_epi_task_P1,dqnb_epi_task_P2,dqnb_epi_task_P3,dqnb_epi_latency_P1,dqnb_epi_latency_P2,dqnb_epi_latency_P3,\
dqnb_epi_energy_P1,dqnb_epi_energy_P2,dqnb_epi_energy_P3,dqnb_epi_num_task_P1,dqnb_epi_num_task_P2,dqnb_epi_num_task_P3,dqnb_epi_task_select_P1,dqnb_epi_task_select_P2,dqnb_epi_task_select_P3,\
dqnb_mean_scores,dqnb_mean_task,dqnb_mean_energy,dqnb_mean_latency,dqnb_win_num_task, dqnb_mean_tasks_P1,dqnb_mean_tasks_P2,dqnb_mean_tasks_P3,\
dqnb_mean_energy_P1,dqnb_mean_energy_P2,dqnb_mean_energy_P3,dqnb_mean_latency_P1,dqnb_mean_latency_P2,dqnb_mean_latency_P3 ,dqnb_win_num_task_P1, dqnb_win_num_task_P2,dqnb_win_num_task_P3,\
dqnb_epi_datasize,dqnb_epi_datasize_P1,dqnb_epi_datasize_P2,dqnb_epi_datasize_P3,dqnb_mean_datasize,dqnb_mean_datasize_P1,dqnb_mean_datasize_P2,dqnb_mean_datasize_P3 = dqnb()

In [ ]:
import csv


# Combining data into a list of tuples
data = zip(range(1, len(dqnb_epi_scores) + 1), dqnb_epi_scores,dqnb_epi_task,dqnb_epi_energy,dqnb_epi_latency,dqnb_epi_num_task,dqnb_epi_task_P1,dqnb_epi_task_P2,
           dqnb_epi_task_P3,dqnb_epi_latency_P1,dqnb_epi_latency_P2,dqnb_epi_latency_P3,
           dqnb_epi_energy_P1,dqnb_epi_energy_P2,dqnb_epi_energy_P3,dqnb_epi_num_task_P1,dqnb_epi_num_task_P2,dqnb_epi_num_task_P3,
           dqnb_epi_datasize,dqnb_epi_datasize_P1,dqnb_epi_datasize_P2,dqnb_epi_datasize_P3,dqnb_epi_task_select_P1,dqnb_epi_task_select_P2,dqnb_epi_task_select_P3)

# Writing data to a CSV file
with open("data_dqnb.csv", mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Index","dqnb_epi_scores","dqnb_epi_task","dqnb_epi_energy","dqnb_epi_latency","dqnb_epi_num_task","dqnb_epi_task_P1","dqnb_epi_task_P2","dqnb_epi_task_P3",
                     "dqnb_epi_latency_P1","dqnb_epi_latency_P2","dqnb_epi_latency_P3","dqnb_epi_energy_P1","dqnb_epi_energy_P2","dqnb_epi_energy_P3","dqnb_epi_num_task_P1",
                     "dqnb_epi_num_task_P2","dqnb_epi_num_task_P3","dqnb_epi_datasize","dqnb_epi_datasize_P1","dqnb_epi_datasize_P2"
                     "dqnb_epi_datasize_P3","dqnb_epi_task_select_P1","dqnb_epi_task_select_P2","dqnb_epi_task_select_P3"])  # Writing header
    writer.writerows(data)  # Writing data rows

In [ ]:
import numpy as np
from mec_environment import MECEnvironment
from random_agent import RandomAgent

mec_evn = MECEnvironment(user_cnt=50, mec_cnt=3,Env_Type=2)

In [ ]:
action_size = mec_evn.action_size()
Task=mec_evn.get_task()
states = mec_evn.get_state(Task)
state_size = mec_evn.state_size()
action_space = mec_evn.actions
print('Action size: {}'.format(action_size))
print('Action space: {}'.format(action_space))
print('Observes a state with length: {}'.format(state_size))
print('state size: {}'.format(len(states)))
print('The state for the first agent looks like:', states)

In [ ]:
epoch_no =2500
window_size =250

In [ ]:
import torch
from collections import deque
from greedy_agent import GreedyAgent
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
greedy_agent = GreedyAgent(action_size=action_size)

In [ ]:
def greedy(n_episodes=epoch_no, max_t=1000, eps_start=1.0, eps_end=0.001, eps_decay=0.995,bench_score=1500000):
    """Deep Q-Learning.

    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of atimesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    epi_scores = []                        # list containing scores from each episode
    epi_task=[]
    epi_task_P1=[]
    epi_task_P2=[]
    epi_task_P3=[]


    epi_latency=[]
    epi_latency_P1=[]
    epi_latency_P2=[]
    epi_latency_P3=[]
    epi_task_select_P1=[]
    epi_task_select_P2=[]
    epi_task_select_P3=[]
    epi_datasize=[]
    epi_datasize_P1=[]
    epi_datasize_P2=[]
    epi_datasize_P3=[]

    epi_energy=[]
    epi_energy_P1=[]
    epi_energy_P2=[]
    epi_energy_P3=[]

    epi_num_task=[]
    epi_num_task_P1=[]
    epi_num_task_P2=[]
    epi_num_task_P3=[]

    mean_scores = [] # average score by the windos size
    mean_task=[] # average task ratio by window size
    mean_energy = [] # average energy cost by window size
    mean_latency=[]
    mean_datasize=[]

    mean_tasks_P1=[]
    mean_tasks_P2=[]
    mean_tasks_P3=[]

    mean_energy_P1=[]
    mean_energy_P2=[]
    mean_energy_P3=[]

    mean_latency_P1=[]
    mean_latency_P2=[]
    mean_latency_P3=[]


    mean_datasize_P1=[]
    mean_datasize_P2=[]
    mean_datasize_P3=[]
    reward_list_P1=[]
    reward_list_P2=[]
    reward_list_P3=[]

    scores_window = deque(maxlen=window_size)  # last score_window_size scores
    task_done_ratio_window = deque(maxlen=window_size)  # last  task dones
    energy_window = deque(maxlen=window_size)  # last  engery cost
    latency_window = deque(maxlen=window_size)  # last  engery cost
    datasize_window= deque(maxlen=window_size)


    mean_tasks_P1_window = deque(maxlen=window_size)  # last  engery cost
    mean_tasks_P2_window = deque(maxlen=window_size)  # last  engery cost
    mean_tasks_P3_window = deque(maxlen=window_size)  # last  engery cost

    win_num_task = deque(maxlen=window_size)
    win_num_task_P1 = deque(maxlen=window_size)  # last  engery cost
    win_num_task_P2 = deque(maxlen=window_size)  # last  engery cost
    win_num_task_P3 = deque(maxlen=window_size)  # last  engery cost

    mean_datasize_window=deque(maxlen=window_size)
    mean_datasize_P1_window= deque(maxlen=window_size)  # last  engery cost
    mean_datasize_P2_window = deque(maxlen=window_size)  # last  engery cost
    mean_datasize_P3_window = deque(maxlen=window_size)  # last  engery cost

    mean_energy_window_P1 = deque(maxlen=window_size)  # last  engery cost
    mean_energy_window_P2 = deque(maxlen=window_size)  # last  engery cost
    mean_energy_window_P3 = deque(maxlen=window_size)  # last  engery cost

    mean_latency_P1_window=deque(maxlen=window_size)
    mean_latency_P2_window=deque(maxlen=window_size)
    mean_latency_P3_window=deque(maxlen=window_size)



    eps = eps_start                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        mec_evn.reset() # reset the environment

        score = 0
        energy = []
        latency=[]
        task_done = []
        energy_P1=[]
        energy_P2=[]
        energy_P3=[]
        task_done_P1=[]
        task_done_P2=[]
        task_done_P3=[]
        latency_P1=[]
        latency_P2=[]
        latency_P3=[]
        datasize=[]
        datasize_P1=[]
        datasize_P2=[]
        datasize_P3=[]



        tasks = mec_evn.get_task()
        for t in range(max_t):

            state = mec_evn.get_state(tasks)
            max_reward_task,max_reward_server,max_reward_frequency = greedy_agent.action(tasks,mec_evn)
            greedy_action=[(max_reward_server,max_reward_frequency)]
            #greedy_task=mec_evn.priority_queues[max_reward_task].pop(0)

#             print("action=",action)
            next_state, feedback,done,action_task = mec_evn.greedy_step(greedy_action,tasks,max_reward_task)




            score += feedback['reward']
            energy.append(feedback['energy'])
            datasize.append(action_task['data_size'])
            if feedback['task_done']==1:

              latency.append(feedback['latency'])


            if action_task['data_size'] != 0:
              task_done.append(feedback['task_done'])
              if action_task['priority']==1:
                  energy_P1.append(feedback['energy'])
                  task_done_P1.append(feedback['task_done'])
                  datasize_P1.append(action_task['data_size'])
                  reward_list_P1.append(feedback['reward'])

                  if feedback['task_done']:
                    latency_P1.append(feedback['latency'])

              if action_task['priority']==2:
                  energy_P2.append(feedback['energy'])
                  task_done_P2.append(feedback['task_done']*feedback['task_done'])
                  datasize_P2.append(action_task['data_size'])
                  reward_list_P2.append(feedback['reward'])
                  if feedback['task_done']:
                    latency_P2.append(feedback['latency'])

              if action_task['priority']==3:
                  energy_P3.append(feedback['energy'])
                  task_done_P3.append(feedback['task_done'])
                  datasize_P3.append(action_task['data_size'])
                  reward_list_P3.append(feedback['reward'])
                  if feedback['task_done']:
                    latency_P3.append(feedback['latency'])


            tasks = mec_evn.get_task()

            if done:
                break
        scores_window.append(score)       # save most recent score
        epi_scores.append(score)              # save most recent score
        mean_score = np.mean(scores_window)
        epi_task.append(np.mean(task_done))
        epi_task_P1.append(np.mean(task_done_P1))
        epi_task_P2.append(np.mean(task_done_P2))
        epi_task_P3.append(np.mean(task_done_P3))

        epi_task_select_P1.append(len(task_done_P1))
        epi_task_select_P2.append(len(task_done_P2))
        epi_task_select_P3.append(len(task_done_P3))

        epi_latency.append(np.mean(latency))
        epi_latency_P1.append(np.mean(latency_P1))
        epi_latency_P2.append(np.mean(latency_P2))
        epi_latency_P3.append(np.mean(latency_P3))

        epi_energy.append(np.mean(energy))
        epi_energy_P1.append(np.mean(energy_P1))
        epi_energy_P2.append(np.mean(energy_P2))
        epi_energy_P3.append(np.mean(energy_P3))

        epi_num_task.append(sum(task_done_P1)+sum(task_done_P2)+sum(task_done_P3))
        epi_num_task_P1.append(sum(task_done_P1))
        epi_num_task_P2.append(sum(task_done_P2))
        epi_num_task_P3.append(sum(task_done_P3))

        epi_datasize.append(np.mean(datasize))
        epi_datasize_P1.append(np.mean(datasize_P1))
        epi_datasize_P2.append(np.mean(datasize_P2))
        epi_datasize_P3.append(np.mean(datasize_P3))





        eps = max(eps_end, eps_decay*eps) # decrease epsilon

        print('\rEpisode {}\tAverage Score: {:.4f}\tmeantaskP1: {:.4f}\tmeantaskP2: {:.4f}\tmeantaskP3: {:.4f}\tmeanenergyP1: \
            {:.4f}\tmeanenergyP2: {:.4f}\tmeanenergyP3: {:.4f}\tmeanlatencyP1: {:.4f}\tmeanlatencyP2: {:.4f}\tmeanlatencyP3:{:.4f}'.format(i_episode, \
            mean_score, np.mean(task_done_P1), np.mean(task_done_P2), np.mean(task_done_P3), np.mean(energy_P1), np.mean(energy_P2), np.mean(energy_P3), \
                                    np.mean(latency_P1), np.mean(latency_P2), np.mean(latency_P3)), end="")





        if i_episode % window_size == 0:


            mean_scores.append(mean_score)
            task_done_ratio_window.append(np.mean(task_done))       # task done ratio
            mean_task.append(np.mean(task_done_ratio_window))
            energy_window.append(np.mean(energy))
            latency_window.append(np.mean(latency))

            mean_energy.append(np.mean(energy_window))
            mean_latency.append(np.mean(latency_window))


            mean_datasize_window.append(np.mean(datasize))
            mean_datasize.append(np.mean(mean_datasize_window))

            mean_tasks_P1_window.append(np.mean(task_done_P1))
            mean_tasks_P1.append(np.mean( mean_tasks_P1_window))

            mean_tasks_P2_window.append(np.mean(task_done_P2))
            mean_tasks_P2.append(np.mean( mean_tasks_P2_window))

            mean_tasks_P3_window.append(np.mean(task_done_P3))
            mean_tasks_P3.append(np.mean( mean_tasks_P3_window))

            mean_energy_window_P1.append(np.mean(energy_P1))
            mean_energy_P1.append(np.mean(mean_energy_window_P1))

            mean_energy_window_P2.append(np.mean(energy_P2))
            mean_energy_P2.append(np.mean(mean_energy_window_P2))

            mean_energy_window_P3.append(np.mean(energy_P3))
            mean_energy_P3.append(np.mean(mean_energy_window_P3))


            mean_latency_P1_window.append(np.mean(latency_P1))
            mean_latency_P1.append(np.mean( mean_latency_P1_window))

            mean_latency_P2_window.append(np.mean(latency_P2))
            mean_latency_P2.append(np.mean( mean_latency_P2_window))

            mean_latency_P3_window.append(np.mean(latency_P3))
            mean_latency_P3.append(np.mean( mean_latency_P3_window))

            mean_datasize_P1_window.append(np.mean(datasize_P1))
            mean_datasize_P1.append(np.mean(mean_datasize_P1_window))

            mean_datasize_P2_window.append(np.mean(datasize_P2))
            mean_datasize_P2.append(np.mean(mean_datasize_P2_window))

            mean_datasize_P3_window.append(np.mean(datasize_P3))
            mean_datasize_P3.append(np.mean(mean_datasize_P3_window))


            win_num_task.append(sum(task_done_P1)+sum(task_done_P2)+sum(task_done_P3))
            win_num_task_P1.append(sum(task_done_P1))
            win_num_task_P2.append(sum(task_done_P2))
            win_num_task_P3.append(sum(task_done_P3))


            print('\rEpisode {}\tAverage Score: {:.4f}\tmeantaskP1: {:.4f}\tmeantaskP2: {:.4f}\tmeantaskP3: {:.4f}\tmeanenergyP1: \
            {:.4f}\tmeanenergyP2: {:.4f}\tmeanenergyP3: {:.4f} \tmeanlatencyP1: {:.4f} \tmeanlatencyP2: {:.4f} \tmeanlatencyP3: {:.4f} \tmeandatasize: {:.4f} \tmeandatasizeP1: {:.4f} \tmeandatasizeP2: {:.4f} \tmeandatasizeP3: {:.4f}' .format(i_episode, mean_score,\
            mean_tasks_P1[-1],mean_tasks_P2[-1],mean_tasks_P3[-1],mean_energy_P1[-1],mean_energy_P2[-1],mean_energy_P3[-1], mean_latency_P1[-1],mean_latency_P2[-1],mean_latency_P3[-1],mean_datasize[-1],mean_datasize_P1[-1], \
                                                                                                                          mean_datasize_P2[-1],mean_datasize_P3[-1]



                                                                                                                                                                                                                                                       ))


            #print(len(task_done_P1),len(task_done_P2),len(task_done_P3))
           # print(len(task_done_P1)+len(task_done_P2)+len(task_done_P3))
            print('mean_task_till_this_window',np.mean(epi_task[-window_size:]), 'T1::',   np.mean(epi_task_P1[-window_size:]) ,'T2::' ,np.mean(epi_task_P2[-window_size:]) ,'T3::',np.mean(epi_task_P3[-window_size:]))
            #print('mean_energy_till_this_window',np.mean(mean_energy[-window_size:]), 'T1::', np.mean(mean_energy_P1[-window_size:])  ,'T2::', np.mean(mean_energy_P2[-window_size:])  ,'T3::',np.mean(mean_energy_P3[-window_size:]))
           # print('mean Latency till this window',np.mean(mean_latency[-window_size:]),'T1::', np.mean(mean_latency_P1[-window_size:]),'T1::', np.mean(mean_latency_P2[-window_size:]),'T1::', np.mean(mean_latency_P3[-window_size:]))
           # print(type(mean_tasks_P1),type(epi_num_task_P1))
            #print('mean_datasize', mean_datasize[-1])
            print( 'task_select_P1', np.mean(epi_task_select_P1[-window_size:]), 'task_select_P2', np.mean(epi_task_select_P2[-window_size:]),'task_select_P3', np.mean(epi_task_select_P3[-window_size:]))
            print('total_task_done_P1::', np.mean(epi_num_task_P1[-window_size:]),'total_task_done_P2::', np.mean(epi_num_task_P2[-window_size:]),'total_task_done_P3::',np.mean(epi_num_task_P3[-window_size:]))

            #print(np.mean(reward_list_P1), np.mean(reward_list_P2), np.mean(reward_list_P3))
            #print(min(reward_list_P1), min(reward_list_P2), min(reward_list_P3))
           # print(max(reward_list_P1), max(reward_list_P2), max(reward_list_P3))


            #print(mean_tasks_P1)
           # print(epi_task_P1)





    return epi_scores,epi_task,epi_energy,epi_latency,epi_num_task, epi_task_P1,epi_task_P2,epi_task_P3,epi_latency_P1,epi_latency_P2,epi_latency_P3,epi_energy_P1,epi_energy_P2,epi_energy_P3,epi_num_task_P1,epi_num_task_P2,epi_num_task_P3, \
    epi_task_select_P1,epi_task_select_P2,epi_task_select_P3,mean_scores,mean_task,mean_energy,mean_latency,win_num_task, mean_tasks_P1,mean_tasks_P2,mean_tasks_P3, mean_energy_P1,\
                 mean_energy_P2, mean_energy_P3,mean_latency_P1,mean_latency_P2,mean_latency_P3,win_num_task_P1,win_num_task_P2,win_num_task_P3,epi_datasize,epi_datasize_P1,epi_datasize_P2,epi_datasize_P3,\
                 mean_datasize,mean_datasize_P1,mean_datasize_P2,mean_datasize_P3

greedy_epi_scores,greedy_epi_task,greedy_epi_energy,greedy_epi_latency,greedy_epi_num_task,greedy_epi_task_P1,greedy_epi_task_P2,greedy_epi_task_P3,greedy_epi_latency_P1,greedy_epi_latency_P2,greedy_epi_latency_P3,\
greedy_epi_energy_P1,greedy_epi_energy_P2,greedy_epi_energy_P3,greedy_epi_num_task_P1,greedy_epi_num_task_P2,greedy_epi_num_task_P3,greedy_epi_task_select_P1,greedy_epi_task_select_P2,greedy_epi_task_select_P3,\
greedy_mean_scores,greedy_mean_task,greedy_mean_energy,greedy_mean_latency,greedy_win_num_task, greedy_mean_tasks_P1,greedy_mean_tasks_P2,greedy_mean_tasks_P3,\
greedy_mean_energy_P1,greedy_mean_energy_P2,greedy_mean_energy_P3,greedy_mean_latency_P1,greedy_mean_latency_P2,greedy_mean_latency_P3 ,greedy_win_num_task_P1, greedy_win_num_task_P2,greedy_win_num_task_P3,\
greedy_epi_datasize,greedy_epi_datasize_P1,greedy_epi_datasize_P2,greedy_epi_datasize_P3,greedy_mean_datasize,greedy_mean_datasize_P1,greedy_mean_datasize_P2,greedy_mean_datasize_P3 = greedy()

In [ ]:
import csv


# Combining data into a list of tuples
data = zip(range(1, len(greedy_epi_scores) + 1), greedy_epi_scores,greedy_epi_task,greedy_epi_energy,greedy_epi_latency,greedy_epi_num_task,greedy_epi_task_P1,greedy_epi_task_P2,
           greedy_epi_task_P3,greedy_epi_latency_P1,greedy_epi_latency_P2,greedy_epi_latency_P3,
           greedy_epi_energy_P1,greedy_epi_energy_P2,greedy_epi_energy_P3,greedy_epi_num_task_P1,greedy_epi_num_task_P2,greedy_epi_num_task_P3,
           greedy_epi_datasize,greedy_epi_datasize_P1,greedy_epi_datasize_P2,greedy_epi_datasize_P3,greedy_epi_task_select_P1,greedy_epi_task_select_P2,greedy_epi_task_select_P3)

# Writing data to a CSV file
with open("data_greedy.csv", mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Index","greedy_epi_scores","greedy_epi_task","greedy_epi_energy","greedy_epi_latency","greedy_epi_num_task","greedy_epi_task_P1","greedy_epi_task_P2","greedy_epi_task_P3",
                     "greedy_epi_latency_P1","greedy_epi_latency_P2","greedy_epi_latency_P3","greedy_epi_energy_P1","greedy_epi_energy_P2","greedy_epi_energy_P3","greedy_epi_num_task_P1",
                     "greedy_epi_num_task_P2","greedy_epi_num_task_P3","greedy_epi_datasize","greedy_epi_datasize_P1","greedy_epi_datasize_P2"
                     "greedy_epi_datasize_P3","greedy_epi_task_select_P1","greedy_epi_task_select_P2","greedy_epi_task_select_P3"])  # Writing header
    writer.writerows(data)  # Writing data rows
